<a href="https://colab.research.google.com/github/jeonggunlee/CUDATeaching/blob/master/hello_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **한국로봇산업진흥원: GPU 기반의 고성능 CUDA 프로그래밍**
## 한림대학교 소프트웨어 융합대학 이정근
## 2019년 2월

안녕하세요. 반갑습니다. 한림대학교 소프트웨어융합대학 이정근 교수라고 합니다.
이번 GPU CUDA 교육에 참여해 주셔서 감사합니다. 본 강의는 이론과 실습으로 이루어져 있으며, 실습은 Colab을 통해서 진행될 예정입니다.

우선 교육에 참여하신 모든 선생님들께서는 Github 및 Colab 계정을 만들어 주시면 감사드리겠습니다.

* Github 주소: www.github.com
* Colab 주소: colab.research.google.com

*  *  *

Colab에서 CUDA Coding을 실습하기 위해서는 몇가지 사항을 알아야합니다.

* [코드 셀]에서 command-line 명령어 실행시키기
   - !ls : 현재 디렉토리의 내용을 보여준다.
   - %cd *dir*: *dir* 디렉토리로 이동한다.
   - %pwd: 현재 위치한 디렉토리 위치를 보여준다.
   - !git: git 명령어를 실행시킨다.

* 또한 GPU 장치와 관련되어 다음 명령어를 확인해주시기 바랍니다.
   - !nvidia-smi: 현재 사용하고 있는 GPU의 스펙과 작동 상황을 보여준다.
   - !nvcc: Nvidia CUDA Compiler를 실행시킨다.
   
위의 명령어들을 실행 시켜 보시기 바랍니다. 


처음 명령어로 pwd 명령어를 실행시켜보도록 하겠습니다. pwd는 사용자가 위치한 디렉토리를 보여줍니다.

In [3]:
!pwd

/content


현재의 디렉토리가 content에 위치하고 있네요. 기본적으로 colab을 사용하는 사용자는 항상 위와 같은 위치를 초기 디렉토리로 사용하게 됩니다.
다음으로 Unix/Linux 계열의 가장 기본적인 명령인 ls를 사용해보도록 하겠습니다. ls 명령어는 현재 디렉토리에 있는 화일 또는 디렉토리를 보여줍니다.

In [2]:
!ls

sample_data


위에서 보는 바와 같이 sample_data라는 디렉토리가 하위 디렉토리에 있다는 것을 알 수 있습니다.

다음으로 우리가 사용하고 있는 시스템의 프로세서 및 GPU에 대해서 알아보도록 하지요.
우선 CPU에 대한 정보를 얻기 위해서 Proc file 시스템의 cpuinfo를 cat 명령어로 보도록 해보지요.

In [4]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: G

자 다음엔 GPU의 스펙을 살펴보도록 하지요. GPU에 대한 정보를 얻기 위해서는 nvidia-smi 명령어를 사용합니다.

In [5]:
!nvidia-smi

Mon Jan 28 05:41:45 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

CPU와 GPU에 대한 정보를 확인 한 후에는 우리가 수행하고자 하는 코드들을 github로 부터 가져와서 시행해보도록 하겠습니다.

저같은 경우는 https://github.com/jeonggunleee/CUDATeaching 의 내용을 git 명령어를 통하여 clone 해오도록 하겠습니다.


In [6]:
!git clone https://github.com/jeonggunlee/CUDATeaching

Cloning into 'CUDATeaching'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 154 (delta 7), reused 0 (delta 0), pack-reused 139
Receiving objects: 100% (154/154), 21.17 MiB | 8.26 MiB/s, done.
Resolving deltas: 100% (57/57), done.


Git clone이 완료된 후에 올바로 clone이 되었는지 확인하기 위하여 ls 명령어를 수행해보독 하겠습니다.

In [7]:
!ls

CUDATeaching  sample_data


이후, cd 명령어를 이용하여 CUDATeaching 디렉토리로 들어가보도록 하겠습니다. 주의 할점은 시스템 명령어를 시행할때 ! 또는 %를 붙여주어야 한다는 것입니다.

In [8]:
%cd CUDATeaching

/content/CUDATeaching


다시 CUDATeaching 디렉토리로 들어온 후에 ls 명령어를 통하여 디렉토리에 어떤 화일이 있는지 살펴보도록 하겠습니다.

In [9]:
!ls

'00_병렬 컴퓨팅 소개.pdf'   colab_test.ipynb   hello_CUDA.ipynb
 01_cuda_lab		    gpu4cloud.md       README.md


In [0]:
위 화일 및 디렉토리 중,01_cuda_lab에 들어가 보도록 하겠습니다. 01_cuda_lab에는 실습을 위한 cuda 소스 코드들이 있습니다.

In [10]:
%cd 01_cuda_lab

/content/CUDATeaching/01_cuda_lab


01_cuda_lab에 들어온후에 ls 명령어를 통하여 어떤 화일들이 있는지 확인해보록 하겠습니다.

In [11]:
!ls

00_reduction_all.cu	     06_matmul_tile3_noBankConflict.cu	atomicAdd.cu
00_reduction_final.cu	     07_matmul_tile4_unroll.cu		bankConflict.cu
01_reduction.cu		     1hello.cu				clock.cu
02_reduction.cu		     2Dindex.cu				CUDA_lab.txt
03_matmul.cu		     2index.cu				globalMemory.cu
04_matmul_tile.cu	     a.out				README.md
05_matmul_tile2_mem_coel.cu  async_streams.cu			transpose.cu


가장 기본 적인 코드로 1hello.cu 를 컴파일 하고 실행시켜보도록 하겠습니다.
1hello.cu의 내용은 아래와 같습니다.

```cuda
#include <stdio.h>

__global__ void helloCUDA(void)
{
  printf("Hello thread %d in block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{
  helloCUDA<<<3, 4>>>();
  cudaDeviceReset();
  return 0;
}
```


1hello.cu를 컴파일하기 위하여 nvdia GPU 컴파일러 명령어인 nvcc를 사용합니다.


In [0]:
!nvcc ./1hello.cu

이후, 컴파일된 실행화일인 a.out을 실행시키면 해당 결과가 프린트 됩니다.

In [14]:
!./a.out

Hello thread 0 in block 0
Hello thread 1 in block 0
Hello thread 2 in block 0
Hello thread 3 in block 0
Hello thread 0 in block 2
Hello thread 1 in block 2
Hello thread 2 in block 2
Hello thread 3 in block 2
Hello thread 0 in block 1
Hello thread 1 in block 1
Hello thread 2 in block 1
Hello thread 3 in block 1


오늘 colaboratory를 활용하여 간단히 GPU CUDA Programming을 하는 방법을 보여드렸습니다. 잘 활용하시면 좋을 것 같네요!
감사합니다.
